In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
file_name='parser_data.csv'

In [3]:
spark = SparkSession \
    .builder \
    .appName("PySpark") \
    .getOrCreate()

In [4]:
metrics = [
    'included_count',
    'received_count',
    'early_count', 
    'in_count', 
    'outside_count', 
    'cancelled_count', 
    'portable_count',
    'stationary_count', 
    'invalid_count', 
    'valid_count', 
    'lost_count', 
    'unaccounted_count'
]
candidates = [
    'Baburin', 
    'Grudinin', 
    'Zhirinovsky',
    'Putin', 
    'Sobchak', 
    'Suraykin', 
    'Titov',
    'Yavlinsky'
]
df = spark.read.csv(file_name, inferSchema=True, header=True).toDF(
    'region','tik', 'uik', *metrics, *candidates
)

###  Найти явку (%) по всем регионам, результат отсортировать по убыванию

In [5]:
percent_df = df.groupBy('region') \
        .agg(F.sum('included_count').alias('sum_included_count'), \
            F.sum('valid_count').alias('sum_valid_count')) \
        .withColumn('percent_of', F.col('sum_valid_count') / F.col('sum_included_count') * 100) \
        .select('region', 'percent_of') \
        .sort('percent_of', ascending=False)
percent_df.show(10)

+--------------------+-----------------+
|              region|       percent_of|
+--------------------+-----------------+
|Территория за пре...|96.81955215029434|
|     Республика Тыва|92.96295873262441|
|Кабардино-Балкарс...|91.57903302417913|
|Ямало-Ненецкий ав...|91.16667520623047|
|Чеченская Республика| 91.1574260006905|
|Республика Северн...|89.26346725367218|
| Республика Дагестан|87.05958846102465|
|Карачаево-Черкесс...|86.96679462111899|
| Кемеровская область|82.54406588112312|
|Республика Ингушетия|81.63562162285402|
+--------------------+-----------------+
only showing top 10 rows



### Выбрать любимого кандидата и найти тот избиратльный участок, на котором он получил наибольший результат (учитывать участки на которых проголосовало больше 300 человек)

In [6]:
favorite = 'Putin'

In [7]:
df_favorite = df.filter(F.col(favorite) >= 300) \
        .select('region','tik','uik', 'included_count', favorite) \
        .withColumn('percent_of', F.col(favorite) / F.col('included_count') * 100) \
        .select('region','tik','uik', favorite, 'percent_of') \
        .sort('percent_of', ascending=False)
df_favorite.show(10)

+--------------------+--------------------+--------+-----+-----------------+
|              region|                 tik|     uik|Putin|       percent_of|
+--------------------+--------------------+--------+-----+-----------------+
|     Республика Тыва|      Бай-Тайгинская| УИК №43|  760|            100.0|
|     Республика Тыва|        Улуг-Хемская|УИК №159|  358|            100.0|
|     Республика Тыва|           Эрзинская|УИК №180|  433|            100.0|
| Кемеровская область|Прокопьевск, Цент...|УИК №876|  732|            100.0|
|Республика Татарс...|         Апастовская|УИК №909|  423|99.76415094339622|
|     Республика Тыва|         Тес-Хемская|УИК №145|  370|99.73045822102425|
|     Республика Тыва|      Бай-Тайгинская| УИК №45|  731|99.72714870395635|
|     Республика Тыва|        Улуг-Хемская|УИК №165|  878| 99.6594778660613|
|     Республика Тыва|        Улуг-Хемская|УИК №163|  815|99.63325183374083|
|Республика Башкор...|    Уфа, Калининская| УИК №98|  456|99.56331877729258|

### Найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна

In [8]:
df_reg = df.groupBy('region','tik') \
        .agg(F.sum('included_count').alias('sum_included_count'), \
            F.sum('valid_count').alias('sum_valid_count')) \
        .withColumn('percent_of', F.col('sum_valid_count') / F.col('sum_included_count') * 100) \
        .groupBy('region') \
        .agg((F.max('percent_of') - F.min('percent_of')).alias('diff')) \
        .sort('diff', ascending=False)
df_reg.show(10)

+--------------------+------------------+
|              region|              diff|
+--------------------+------------------+
|Архангельская обл...| 49.58829163166267|
| Сахалинская область|  47.8670553116025|
| Республика Дагестан| 47.37946734555504|
| Республика Калмыкия|43.211829960607844|
|   Самарская область| 41.87282511134117|
|   Красноярский край| 41.33462602081461|
| Саратовская область| 39.82477785287487|
|Республика Татарс...|39.601354722670806|
|  Краснодарский край| 36.48401530541677|
|Республика Адыгея...| 36.26433374911393|
+--------------------+------------------+
only showing top 10 rows



### Посчитать дисперсию по явке для каждого региона (учитывать УИК)

In [9]:
df_disp = df \
    .withColumn('percent_of', F.col('valid_count') / F.col('included_count')) \
    .groupBy('region') \
    .agg(F.stddev('percent_of').alias('std')) \
    .sort('std', ascending=False) 

df_disp.show(10)

+--------------------+-------------------+
|              region|                std|
+--------------------+-------------------+
| Сахалинская область| 0.2019312853758835|
|     Камчатский край|0.16896820928370715|
|     Приморский край|0.16717013657528354|
|  Мурманская область|0.16697864277648017|
|Республика Адыгея...| 0.1637267331819065|
| Республика Калмыкия|0.16033073351999283|
|   Самарская область| 0.1557678807994638|
| Саратовская область| 0.1550544694418556|
| Воронежская область|0.15043667103903102|
|Республика Татарс...|0.14434297889714248|
+--------------------+-------------------+
only showing top 10 rows



### Для каждого кандидата посчитать таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

In [10]:
for candidate in candidates:  
    print(candidate + " results:")
    df.withColumn('percent_of', F.round(F.col(candidate) / F.col('valid_count') * 100)) \
        .groupBy('percent_of') \
        .agg(F.count('uik').alias('uik_count')) \
        .sort(F.col('percent_of'), ascending=False) \
        .show()

Baburin results:
+----------+---------+
|percent_of|uik_count|
+----------+---------+
|      44.0|        1|
|      23.0|        1|
|      21.0|        2|
|      20.0|        2|
|      18.0|        2|
|      17.0|        1|
|      15.0|        2|
|      14.0|        2|
|      13.0|        1|
|      12.0|        1|
|      11.0|        6|
|      10.0|        9|
|       9.0|       10|
|       8.0|       21|
|       7.0|       21|
|       6.0|       37|
|       5.0|       86|
|       4.0|      183|
|       3.0|      577|
|       2.0|     3889|
+----------+---------+
only showing top 20 rows

Grudinin results:
+----------+---------+
|percent_of|uik_count|
+----------+---------+
|      80.0|        1|
|      78.0|        1|
|      75.0|        1|
|      71.0|        1|
|      70.0|        1|
|      69.0|        2|
|      67.0|        3|
|      61.0|        1|
|      60.0|        1|
|      58.0|        3|
|      57.0|        6|
|      56.0|        4|
|      55.0|        3|
|      54.0|       